In [ ]:
#Schema at https://dbdiagram.io/d/63223c6e0911f91ba5b12259

In [ ]:
# Importing sqlalchemy
import sqlalchemy as sql

In [ ]:
# Some more imports.
import os
import sys
from sqlalchemy import Column, ForeignKey, Integer, String, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [ ]:
Base = declarative_base()

class Pokemons(Base):
    __tablename__ = 'pokemons'
    id = Column(Integer, primary_key=True)
    pokemon_name = Column(String(50), nullable=False)
    total = Column(Integer, nullable=False)
    hp = Column(Integer, nullable=False)
    attack = Column(Integer, nullable=False)
    defense = Column(Integer, nullable=False)
    sp_attack = Column(Integer, nullable=False)
    sp_defense = Column(Integer, nullable=False)
    speed = Column(Integer, nullable=False)
    legendary = Column(Boolean, nullable=False)

    type_1 = Column(Integer, ForeignKey('types.id'))
    type_2 = Column(Integer, ForeignKey('types.id'))
    generation = Column(Integer,ForeignKey('generations.id'), nullable=False)

    # I DON'T KNOW WHAT THESE DO =/
    # types = relationship("Types", back_populates="pokemons")
    # generations = relationship("Generations", back_populates="pokemons")

class Generations(Base):
    __tablename__ = 'generations'
    id = Column(Integer, primary_key=True)
    generation = Column(Integer, nullable=False)
    #I am not sure about these....
    # pokemons = relationship("Pokemons", back_populates="generations")

class Types(Base):
    __tablename__ = 'types'
    id = Column(Integer, primary_key=True)
    type_name = Column(String(50), nullable=False)
    #I am not sure about these....
    # pokemons = relationship("Pokemons", back_populates="types")


# Create an engine that stores data in the local directory's
engine = create_engine('sqlite:///pokemon_shop.db')

# Create all tables in the engine. This is equivalent to "Create Table"
# statements in raw SQL.
Base.metadata.create_all(engine)

In [ ]:
# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)
# A DBSession() instance establishes all conversations with the database
# and represents a "staging zone" for all the objects loaded into the
# database session object. Any change made against the objects in the
# session won't be persisted into the database until you call
# session.commit(). If you're not happy about the changes, you can
# revert all of them back to the last commit by calling
# session.rollback()
session = DBSession()

In [ ]:
#Here we move csv data to our new database.
import csv
filename = 'Pokemons.csv'
#Step One. Retrieve types and generations.
types = []
generations = []

with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    for row in datareader:
        #First row is a header with column names. Skip it
        if row[0] == "#":
            continue
        #Grab type 1 and 2 from a row
        type_1 = row[2]
        type_2 = row[3]
        #Grab generation and transforming it into a number. Add it to list.
        generation = int(row[11])
        generations.append(generation)
        #Because we need unique types - check if the type is already on the list.
        if type_1 not in types:
            types.append(type_1)
        else: continue
        
        if type_2 not in types:
            types.append(type_2)
        else: continue
#Add types and generations to their tables.       
for i in types:
    new_type = Types(type_name=i)
    session.add(new_type)
    session.commit()
for i in list(set(generations)):
    new_generation = Generations(generation=i)
    session.add(new_generation)
    session.commit()

In [ ]:
#Step Two. Retrieve Pokemons.
with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    for row in datareader:
        #Skip header.
        if row[0] == "#":
            continue
        #Grab values. Transform strings to numbers where needed.
        name = row[1]
        type_1 = row[2]
        type_2 = row[3]
        #Because for this table we need foreing keys as numbers - grab them from the list above. Add 1 coz 0 is 1 xD
        type_1_index = types.index(type_1)+1
        type_2_index = types.index(type_2)+1
        total = int(row[4])
        hp = int(row[5])
        attack = int(row[6])
        defense = int(row[7])
        sp_attack = int(row[8])
        sp_defense = int(row[9])
        speed = int(row[10])
        generation = int(row[11])
        #Ternary operator for legendary parameter.
        legendary = False if row[12] == "False" else True
        #Create new pokemon and add it to the database.
        new_pokemon = Pokemons(
            pokemon_name = name,
            total = total,
            hp = hp,
            attack = attack,
            defense = defense,
            sp_attack = sp_attack,
            sp_defense = sp_defense,
            speed = speed,
            legendary = legendary,
            #Indexes go here not values
            type_1 = type_1_index,
            type_2 = type_2_index,
            generation = generation
        )
        session.add(new_pokemon)
        session.commit() 

In [ ]:
#Print all pokemons.
s = sql.select([Pokemons])
rp = session.execute(s)
results = rp.fetchall()
for i in results:
    print(i.Pokemons.pokemon_name, i.Pokemons.type_1, i.Pokemons.type_2, i.Pokemons.total,i.Pokemons.hp, i.Pokemons.attack, i.Pokemons.defense, i.Pokemons.sp_attack, i.Pokemons.sp_defense, i.Pokemons.speed, i.Pokemons.generation, i.Pokemons.legendary)

In [ ]:
#Print all types with indexes.
s = sql.select([Types])
rp = session.execute(s)
results = rp.fetchall()

# for i in results:
#     print(i.Types.type_name, i.Types.id)

In [ ]:
#Print all generations with indexes.
s = sql.select([Generations])
rp = session.execute(s)
results = rp.fetchall()
# for i in results:
#     print(i.Generations.generation, i.Generations.id)


In [ ]:



# Example with a where statement.
s = sql.select([Generations]).where(Generations.id == 1)
rp = session.execute(s)
results = rp.fetchall()
# results

s = "SELECT * FROM pokemons"
rp = session.execute(s)
results = rp.fetchall()
# results

s = "SELECT pokemon_name, attack, type_name FROM pokemons, types WHERE types.id = pokemons.type_1 GROUP BY attack"
rp = session.execute(s)
results = rp.fetchall()
# results

In [ ]:
# Notice anything different about the query?
s = session.query(Pokemons)
s = s.filter(Pokemons.id == 1)

# Update example.
s.update({Pokemons.hp: 45})
session.commit()

In [ ]:
#Add one super strong pokemon:
super_pokemon = Pokemons(
            pokemon_name = "Tyranauseras X 3000",
            total = 10000,
            hp = 500,
            attack = 500,
            defense = 500,
            sp_attack = 500,
            sp_defense = 500,
            speed = 500,
            legendary = True,
            #Indexes go here not values
            type_1 = 13,
            type_2 = 10,
            generation = 5
        )
session.add(super_pokemon)
session.commit() 

s = session.query(Pokemons)
s = s.filter(Pokemons.attack == 500)
rp = session.execute(s)
results = rp.fetchall()
for i in results:
    print(i.Pokemons.id, i.Pokemons.pokemon_name, i.Pokemons.type_1, i.Pokemons.type_2, i.Pokemons.total,i.Pokemons.hp, i.Pokemons.attack, i.Pokemons.defense, i.Pokemons.sp_attack, i.Pokemons.sp_defense, i.Pokemons.speed, i.Pokemons.generation, i.Pokemons.legendary)

In [ ]:
# Delete example
s = sql.delete(Pokemons).where(Pokemons.id >= 800)
rp = session.execute(s)

In [ ]:
# Closing session.
session.close()